## Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import warnings
from IPython.display import display
from sklearn.model_selection import train_test_split 
sb.set() 
warnings.filterwarnings('ignore')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Import

In [2]:
dat1 = pd.read_csv("resale-flat-prices-based-on-approval-date-1990-1999.csv")
dat2 = pd.read_csv("resale-flat-prices-based-on-approval-date-2000-feb-2012.csv")
dat3 = pd.read_csv("resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
dat4 = pd.read_csv("resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv")
dat5 = pd.read_csv("resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

In [3]:
# The last two .csv files have extra collumns
dat4 = dat4.drop(columns=['remaining_lease'])
dat5 = dat5.drop(columns=['remaining_lease'])

In [9]:
data = pd.concat([dat1, dat2, dat3, dat4, dat5])
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 826581 entries, 0 to 80373
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                826581 non-null  object 
 1   town                 826581 non-null  object 
 2   flat_type            826581 non-null  object 
 3   block                826581 non-null  object 
 4   street_name          826581 non-null  object 
 5   storey_range         826581 non-null  object 
 6   floor_area_sqm       826581 non-null  float64
 7   flat_model           826581 non-null  object 
 8   lease_commence_date  826581 non-null  int64  
 9   resale_price         826581 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 69.4+ MB


## Data cleaning

In [11]:
# Removing empty spaces from the columns
data.columns = data.columns.str.strip()

# Removing leading and trailing whitespaces
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Removing duplicate rows
data.drop_duplicates(inplace=True)

# Data types of columns
print("\nData types of columns:")
print(data.dtypes)

# Missing values
print("\nMissing values:")
print(data.isnull().sum())

# Dropping the rows with resale_price and floor_area_sqm
data = data.dropna(subset=['floor_area_sqm'])
data = data.dropna(subset=['resale_price'])


Data types of columns:
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
dtype: object

Missing values:
month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
dtype: int64


In [12]:
# Convert the 'month' column to datetime format
# data['month'] = pd.to_datetime(data['month'], format='%Y-%m')

# Extract the year from the 'month' column and create a new column named 'year'
# data['year'] = data['month'].dt.year
data['resale_year'] = pd.to_datetime(data['month'], format='%Y-%m').dt.year

# Save the DataFrame back to a CSV file
data.to_csv('cleaned-Housing.csv', index=False)  # Set index=False to avoid saving the DataFrame index as a column

In [13]:
# Define the lower and upper quantile thresholds
lower_quartile = data['resale_price'].quantile(0.25)
upper_quartile = data['resale_price'].quantile(0.75)
interquartile_range = upper_quartile - lower_quartile
lower_threshold = lower_quartile - 1.5*interquartile_range
upper_threshold = upper_quartile + 1.5*interquartile_range

# Filter out the outliers based on the resale price column
data = data[(data['resale_price'] >= lower_threshold) & (data['resale_price'] <= upper_threshold)]

In [14]:
pd.options.display.max_columns = None
display(data)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,resale_year
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,1990
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,1990
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,1990
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,1990
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,1990
...,...,...,...,...,...,...,...,...,...,...,...
80369,2020-09,YISHUN,5 ROOM,716,YISHUN ST 71,07 TO 09,131.0,Improved,1987,440000.0,2020
80370,2020-09,YISHUN,5 ROOM,760,YISHUN ST 72,07 TO 09,122.0,Improved,1987,458000.0,2020
80371,2020-09,YISHUN,5 ROOM,835,YISHUN ST 81,04 TO 06,122.0,Improved,1987,490000.0,2020
80372,2020-09,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,146.0,Maisonette,1987,558000.0,2020


## Train-test split

In [15]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
train_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,resale_year
19280,2000-08,ANG MO KIO,3 ROOM,103,ANG MO KIO AVE 3,04 TO 06,73.0,New Generation,1978,152000.0,2000
217767,1998-10,SERANGOON,5 ROOM,114,SERANGOON NTH AVE 1,07 TO 09,121.0,IMPROVED,1986,345000.0,1998
235635,2007-06,BEDOK,5 ROOM,669,JLN DAMAI,10 TO 12,125.0,Improved,1996,331000.0,2007
195784,1998-05,YISHUN,3 ROOM,215,YISHUN ST 21,04 TO 06,67.0,NEW GENERATION,1985,135000.0,1998
59663,2019-10,SEMBAWANG,EXECUTIVE,503A,CANBERRA LINK,10 TO 12,130.0,Apartment,2001,525000.0,2019


In [17]:
test_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,resale_year
17905,2012-11,SEMBAWANG,5 ROOM,317,SEMBAWANG VISTA,04 TO 06,121.0,Improved,1999,480000.0,2012
14824,2012-09,YISHUN,4 ROOM,621,YISHUN RING RD,07 TO 09,105.0,Model A,1988,464000.0,2012
20373,2016-03,BEDOK,3 ROOM,122,BEDOK NTH ST 2,07 TO 09,67.0,New Generation,1978,322000.0,2016
3119,1990-03,JURONG WEST,4 ROOM,216,BOON LAY AVE,07 TO 09,88.0,IMPROVED,1975,50000.0,1990
26767,1992-02,BISHAN,3 ROOM,108,BISHAN ST 12,01 TO 03,64.0,SIMPLIFIED,1985,82000.0,1992


## Export

In [18]:
data.to_csv("cleaned-Housing.csv", index=False)
train_data.to_csv("train-Housing.csv", index=False)
test_data.to_csv("test-Housing.csv", index=False)